In [ ]:
from tabulate import tabulate
from itertools import combinations

EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def get_disk_map(input_file_name):
    map = []
    with open(input_file_name, "r") as file:
        for line in file:
            map = [int(c) for c in line.strip().replace("\n", "")]
    return map

In [ ]:
disk_map = get_disk_map(EXAMPLE)
print(disk_map)

In [ ]:
from pydantic import BaseModel

class File(BaseModel):
    id: int
    start: int
    size: int


class FreeSpace(BaseModel):
    start: int
    size: int
    
class BlockMap:
    def __init__(self, files, free_spaces):
        self.files: list[File] = files
        self.free_spaces: list[FreeSpace] = free_spaces

    @classmethod
    def build(cls, disk_map):
        files = []
        free_spaces = []
        current_file_id = 0
        current_position = 0
        for i, d in enumerate(disk_map):
            if i % 2 == 0:
                file_id = current_file_id
                current_file_id += 1
                if d > 0:
                    file = File(id=file_id, start=current_position, size=d)
                    files.append(file)
            else:
                if d > 0:
                    free_space = FreeSpace(start=current_position, size=d)
                    free_spaces = [free_space] + free_spaces
            current_position += d
        return cls(files, free_spaces)

In [ ]:
block_map = BlockMap.build(disk_map)
print(block_map.files, block_map.free_spaces)

In [ ]:
def fill_first_free_space(block_map: BlockMap):
    last_file = block_map.files.pop()
    first_free_space = block_map.free_spaces.pop()
    if first_free_space.size > last_file.size:
        last_file.start = first_free_space.start
        first_free_space.size -= last_file.size
        first_free_space.start += last_file.size
        block_map.free_spaces.append(first_free_space)
        block_map.files = [last_file] + block_map.files
    elif first_free_space.size == last_file.size:
        last_file.start = first_free_space.start
        block_map.files = [last_file] + block_map.files
    elif first_free_space.size < last_file.size:
        new_file = File(
            id=last_file.id, start=first_free_space.start, size=first_free_space.size
        )
        last_file.size -= first_free_space.size
        block_map.files = [new_file] + block_map.files + [last_file]

In [ ]:
fill_first_free_space(block_map)
print(block_map.files, block_map.free_spaces)

In [ ]:
def fill_free_spaces(block_map: BlockMap):
    while block_map.free_spaces and block_map.free_spaces[-1].start < block_map.files[-1].start:
        fill_first_free_space(block_map)


In [ ]:
fill_free_spaces(block_map)

In [ ]:
def calculate_checksum(block_map: BlockMap):
    checksum = 0
    for file in block_map.files:
        for i in range(file.start, file.start + file.size):
            checksum += i*file.id
    return checksum

In [ ]:
def part_1(input_file_name):
    disk_map = get_disk_map(input_file_name)
    block_map = BlockMap.build(disk_map)
    fill_free_spaces(block_map)
    print(calculate_checksum(block_map))

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)